### Step-1: Preliminary setup

In [1]:
#Import gensim library
import gensim
from gensim.models import Word2Vec, Phrases
from gensim.models.phrases import Phraser
from nltk.corpus import PlaintextCorpusReader
import nltk
import pandas as pd
import numpy as np
import time
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter


### Step-2: Load Dataset and Train Word2Vec model

In [2]:
start=time.time()
df = pd.read_csv("./amazon_food_review/Reviews.csv")
df.columns
end=time.time()
print("Time taken %f min"%(((end-start)/ 60)))

Time taken 0.085611 min


### Extract Top 30 Products with the most Reviews

In [3]:
 
top_product = Counter(df.ProductId).most_common()[0:1]

for i in range(len(top_product)):
    if i ==0:
        df2= df[df.ProductId == top_product[i][0]]
    else:
        df2 = pd.concat((df2,df[df.ProductId == top_product[i][0]]))
        
df2.shape

(913, 10)

### Check the count of Reviews in the dataset

In [4]:
from collections import Counter
print(Counter(df2.ProductId).most_common())
print(Counter(df2.Score))

[('B007JFMH8M', 913)]
Counter({5: 624, 4: 217, 3: 56, 2: 12, 1: 4})


### Tokenize the Reviews into Sentences

In [5]:
start=time.time()
#sents=list(df2.Text)
sents = df2.Text
sent_lst = []
for line in sents:
    #print(line)
    for token_line in sent_tokenize(line):
        #print(token_line)
        sent_lst.append([token_line])
end=time.time()
print("Time taken %f min"%(((end-start)/ 60)))

len(sent_lst)

Time taken 0.003627 min


3667

## Sentences - Word_tokenize

In [6]:
from random import shuffle

start=time.time()
sent_lst2 = []
#unigram_lst=[]
for line in sent_lst:
    #print(word_tokenize(line[0]))
    sent_lst2.append(word_tokenize(line[0]))
end=time.time()
print("Time taken %f min"%(((end-start)/ 60)))

shuffle(sent_lst2)
sent_train = sent_lst2[0: round(0.64*(len(sent_lst2)))]
len(sent_train)

Time taken 0.011129 min


2347

### Step-4: hand-write some patterns

In [7]:
#regex grammar
grammar = r"""
JN:{<RB.*|JJ.*>+<.>*}



NP:{<EX|DT|PRP\$>?<RB>*<JN|VB.*>+<NN.*>+<.>*}

cla1a:{<DT|PRP>?<NN.*>+<VB.*>+<JN>+<JJ.*>*<.>*}
        
cla1b:{<cla1a>+<,|CC>+<JN|NN.*>+<JN|NN.*>*<.>*}
        {<cla1a>+<,|CC>+<cla1a>+<.>*}

cla2a:{<DT>?<NN.*|PRP.*>+<IN|VB.*>+<DT>*<NP|NN.*>+<NP|NN.*|JN>*<.>*}

cla2b:{<DT>?<cla2a>+<,|CC>+<DT>*<JN|NN.*|NP>+<.>*}
        {<DT>?<cla2a>+<,|CC>+<cla2a>+<.>*}
        
    
        
cla3: {<DT>?<NN.*>+<NP>+<JN>*<.>*<IN>*<NP>*}

cla4: {<cla1a>+<VB.*>+<JN>*<.>*}

cla4b: {<cla4>+<PRP.*>+<NN.*>+<TO>*<VB.*>*<.>*}


cla5: {<NN.*|PRP.*><NP>+<IN>+<NP>+<.>*}
            {<NN.*|PRP.*><NP>+<IN>+<NP>*<.>*}

cla6: {<PRP>*<MD>*<JN>*<VB.*>+<DT>+<TO>+<NP>+<.>*}

cla7: {<WP>+<PRP>*<MD>+<JN>*<VB.*>+<PRP>+<JN>+<.>*}

cla8: {<PRP>?<MD>*<JN>*<VB.*>+<DT>+<TO>+<PRP\$>+<NN.*>+<.>*}

cla9: {<DT>*<NN>+<MD>*<VB.*>+<DT>+<NN>+<JN>+<IN>+<CD>*<NN.*>*<.>*}

cla10: {<PRP>+<VB.*>+<JN>+<NP>+<IN>+<NP+>*<.>*}

cla11: {<CC>+<DT>*<NN.*>?<JN>+<VB.*>+<IN>+<PRP.*>*<NN>*<.>*}

cla12: {<CC>?<VB.*>+<IN>+<PRP.*>+<.>*}

cla13: {<CC>?<PRP.*>+<VB.*>+<JN>+<IN>+<DT|VB.*>*<JJ.*|JN|NN.*>*<.>*}

cla14:{<CC>?<PRP.*>*<MD>*<JN>*<VB.*><TO|IN>+<VB.*|JN>+<.>*}

cla15: {<CC>?<NN.*|PRP>+<VB.*>+<VB.*|JN>+<PRP.*|DT|WDT|NN>*<VB.*>*<.>*}

cla15b: {<cla15>+<CC>+<JN|JJ>+<.>*}

cla15c: {<cla15>+<TO>+<VB.*>+}

cla16:{<CC>?<DT>+<VB.*>*<JN|JJ.*|NP>+<JN|VB.*>*<.>*}

cla16b:{<cla16><,|CC>+<NN.*|NP>*<.>*}

cla16c:{<cla16b>+<PRP.*>+<JN>*<VB.*>+<.>*}

cla17: {<NP>+<WDT|WP.|WRB>*<PRP.*>*<VB.*>+<NP|JN>*<.>*}

cla17b: {<cla17>+<TO|VBG>+<VB.*>*<JN>*<.>*}

cla17c:{<cla17>+<PRP.*>+<JN | JJ>*}

cla18:{<DT>?<CD>+<IN>*<JN|NN.*>*}

cla18a:{<cla18>+<,|CC>+<cla18>+<.>*}

cla19: {<PRP.*>*<JN>*<WP>?<MD>*<VB.*>+<DT|PRP.*>*<NP|NN.*|PRP.*>+<.>*}

cla19b: {<cla18>+<cla19>+<.>*}

cla20: {<cla15>+<cla12>+<.>*}

cla21: {<PRP>*<MD>*<JN>+<TO>*<VB>+<DT>*<cla16|cla18>*}

cla22: {<VB.*>+<cla2a>+<WRB|WP|WDT>*<PRP>*<VB.*>*<.>*}

cla23:{<cla2a>+<WDT|WP>+<cla19>+}

cla24: {<NNP>+<JN>+<IN>+<DT>+<PRP>*<:|.>*}

cla24b:{<cla24>+<cla17>+}

cla25: {<NN.*>+<TO|PRP.*>*<NN.*>+<,|IN|CC>*<NN.*>*<IN|TO>*<VB.*>+<PRP.*|DT>*<:|.>*}

cla25b:{<cla25>+<,|CC>*<JN|NP>+<.|:>*}

cla26: {<PRP.*>+<NN.*>+<,|IN|CC>*<NN.*>*<VB.*>+<DT|PRP.*>*<JN>*<:|.>*}

cla27: {<cla15>+<IN>*<WRB|WDT|WP.*>*<JN>+<DT|NN.*>*<VB.*>+<:|.>*}

cla28:{<NP|NNP>+<,|CC>*<NP|JN>+}

cla29: {<WP>+<JN>*<PRP.*>+<MD>+<VB.*>*<:|.>*}
    
cla30: {<WP>+<JN>*<VB.*>+<cla21>+<:|.>*}

cla31: {<NN.*>+<,|CC|IN>*<JN|JJ|NN.*>*<,|CC|IN>*<NN.*>*<:|.>*}

cla32: {<NN.*|PRP.*>*<MD>+<VB.*>*<PRP.*|DT>+<JN>*<.|:>*}

cla33:{<cla19>+<DT|JN | RP>*<.|:>*}

cla34:{<PRP.*>+<cla18>+<cla3>*<cla19>+<JN>*<.|:>*}

cla35:{<DT>*<JJS>+<,|CC>*<JJS>*<NN.*><PRP.*|DT>+<JN>*<.|:>*}

cla36:{<cla2a>+<IN>*<DT>*<NN.*|NP|cla31>+<.|:>*}

cla37:{<DT|PRP.*>*<cla15>+<CC|IN|,>*<cla15>+<.|:>*}

cla38:{<DT>+<MD>+<VB.*>+<IN|DT>*<cla31>+<.|:>*}

cla39:{<NN.*|PRP.*>+<VB.*>*<PRP.*>+<JN>*<.|:>*}

cla40:{<NN.*|PRP.*>*<MD>*<VB.*>+<JN|cla16>+<.|:>*}

cla41:{<cla33>+<TO|IN>+<cla2a>+<.|:>*}

cla42:{<cla31>+<TO|IN>+<VB.*>+<TO|IN>+<NP>+<.|:>*}

cla43:{<DT>*<cla31>+<cla40>+<TO|IN>+<DT|PRP.*>*<cla31>*<.|:>*}

cla44:{<WP>+<cla40>+<IN>*<DT|PRP.*>+<NP|NN.*>+<.|:>*}
    
cla44b:{<WP>+<cla40>+<IN>+<cla18|DT|NN.*|cla31>+<.|:>*}

cla45:{<cla2b>+<cla40>+<.|:>*}

cla46:{<cla13>+<NP>+<.|:>*}

cla47:{<WRB>+<JN|VB.*>+<CC|IN>*<JN|VB.*>*<DT|PRP.*>*<VB.*>+<.|:>*}

cla48:{<cla17>+<IN>+<cla2a>+<.|:>*}

cla49:{<cla19>+<IN>+<PRP.*>*<cla18|cla31>+<.|:>*}

cla49b: {<cla49>+<cla3>+<.|:>*}

cla50: {<PRP.*>?<cla12>+<cla31>+<.|:>*}

cla51:{<CC>*<DT|PRP.*>*<cla31>+<cla19+>+<JN>*<.|:>*}

cla52:{<cla2a>+<MD>*<VB.*>+<WRB>*<JN>+<DT>*<PRP.*|cla31>*<VB.*>*<.|:>*}

cla53:{<cla31>+<cla39|VG.*>*<DT|cla31>*<cla33>+<.|:>*}

Quotation:{<:|.>*<\"|\'|\``|\''|\`>+<NN.*|VB.*|JN|NP|JJ|cla24|cla25|cla26|cla27|cla28|cla29|cla30|cla31>*<\"|\'|\``|\''|\`>+<:|.>*}
"""

In [8]:
############# Grammar: NP
#sent = "Nicely smell cookiess"
#sent = "tasty cookies"


############ cla1a
#sent = "The cookiess seemed pretty tasty"
#sent = "the cookiess were excellent"
#sent = "the cookies is very nice"
#sent = "the bottles are not too large"
#sent = "the bottles are too large"
#sent ="the cookies are not good"
#sent ="the cookies are not very good enough"
#sent ="the cookies are not very good"
#sent = "this cookie was surprisingly yummy."

############ cla1b
#sent = "cookies was tasty and soft"
#sent = " the cookies is pretty beautiful and the cookies is pretty soft"
#sent = " the cookies is big and the rasins is very soft"

############ cla2a
#sent="I love the food so much"
#sent = "I participated in a product review"


############ cla2b
#sent = "She likes this big cookie and soft cookie"
#sent = "She likes this cookies and the smell"


############cla3
#sent = "the cookies are way too small"
#sent = "The cookies are soft, chewy, yummy,"
#sent = "These cookies have perfect flavor, excellent moist/dry combination"

############cla4
#sent="the cookie did not smell good"


############ 4b
#sent="please don't waste your money "

############cla5
#sent = "I love cookies with many rasins"


########### 7
#sent= "who would buy it again??"


############ 13
#sent = "It goes down hill from there."
#sent = "They're are so soft, perfect for anytime"


############ 14
#sent = "I can't wait to try "
#sent="can't wait to buy more"

############ 15
#sent="It was great!!!"
#sent = "i love eating it"
#sent = "They're are so soft and perfect"

############ 15b
#sent="They are so soft and tasty!"

############ 15c
#sent = "it is great to buy it"

##########16
#sent = "This is my great best experience!"
#sent = "This is the high point."
#sent = "This was the most delightful cookie i ever tasted."

##########16b
#sent = "the best and freshest cookie"

##########16c
#sent = "the best and freshest cookie I ever had"


##########17
#sent = "this is bad food which hurt your healthy body"
#sent = "The high quality ingredients are a crowd pleaser."


##########17b


##########17c
#sent = "Oatmeal cookies are my favorite"

########## 18
#sent="6g of fat , 27g total carbs"

########## 19
#sent="what's the point?"
#sent="you'll love this snack ....."
#sent = "I highly recommend this cookie"
#sent = "loved the food"
#sent = "loved them"
#sent ="I love my cookies,"

######### 21
#sent = "too old to eat"
#sent="I would never try one again."
#sent="I would never try this one again."

######## 22
#sent="throw them in the garbage which they belong"
#sent="throw them in the garbage where they belong"

######## 23
#sent="I participated in a product review that included a sample"

########25


########25b


########26
#sent="My kids and i  loved these."


########27
#sent="I was surprised how soft the cookie was."
#sent="I was immediately surprised by how soft the cookie was "

########28
#sent = "Delicious and nutritious ....."
#sent = "good price and delicious"

########29
#sent="what else you can say!!!"

######## 30
#sent="what else is there to say...."

####### 31
#sent = "Soft, chewy and tasty."
#sent = "Highly recommend!"
#sent = "yummy and delicious."

####### 32
#sent="I would recommend these"

#######33
#sent = "we ate them all...!!!"
#sent = "we ate them up...!!!"
#sent= "should i buy it again??"

###### 34
#sent="My three year old kid loved it so much !!!"


######36
#sent = "I am a huge fan of the Quaker Chewy cookies!"
#sent = "I am a huge fan of the cookies!"
#sent = "I am a huge fan of the Chewy cookies!"
#sent = "I am a huge fan of Chewy cookies!"
#sent = "I am a huge fan of cookies!"

###### 37
#sent="The moist cookies are satisfying and they are not too sweet."

###### 40
#sent ="I will buy some more"

###### 41
#sent = "I recommend these to anyone with a sweet tooth"


###### 42
#sent = "Great to take as a quick breakfast "

###### 43
#sent = "This cookie would be perfect for breakfast"
#sent = "This cookie would be perfect for my family..."
#sent = "This cookie would be perfect for me"

###### 44
#sent = "What could be better than that?!"

###### 44b
#sent = "What could be better than this one ?!"
#sent = "What could be better than this product ?!"
#sent = "What could be better than these products ?!"
#sent = "What could be better than that?!"

###### 46
#sent = "they are more healthy than most other products."
#sent="they are more healther than other food"

###### 47
#sent = "how soft and fresh it tasted"
#sent="How good it tasted?"

###### 49
#sent="I even shared it with my brother"
#sent="I shared it with my child"

########50
#sent = "They taste like my grandmas homeade cookies.."

########51
#sent = "the kids love them!"
#sent="My  kid loved it so much "

########52
#sent = "I guess the younger folks will like how sweet the cookie is."
#sent=sent_lst[149][0]

########53
#sent="Thank you my mom buying me this!!!"
#sent="thanks for buying this"
#sent="Thanks for buying me this"








#print(sent)
#print()
#print(nltk.pos_tag(nltk.word_tokenize(sent)))
#print()
#sent =  nltk.word_tokenize(sent)
review_lst=[]
for sent in sent_train:
    cp = nltk.RegexpParser(grammar)
    tree = cp.parse(nltk.pos_tag(sent))
    for subtree in tree.subtrees():
        if subtree.label() =="NP":        print("NP", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="JN":         print("JN", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="Quotation":   print("Quotation", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla1a":         print("cla1a", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla1b":         print("cla1b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla2a":         print("cla2a", subtree.leaves()), review_lst.append(subtree.leaves())
        if subtree.label() =="cla2b":         print("cla2b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla3":         print("cla3", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla4":         print("cla4", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla4b":         print("cla4b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla5":         print("cla5", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla6":         print("cla6", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla7":         print("cla7", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla8":         print("cla8", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla9":         print("cla9", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla10":         print("cla10", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla11":         print("cla11", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla12":         print("cla12", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla13":         print("cla13", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla14":        print("cla14", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla15":         print("cla15", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla15b":         print("cla15b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla15c":         print("cla15c", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla16":         print("cla16", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla16b":         print("cla16b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla16c":         print("cla16c", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla17":         print("cla17", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla17b":         print("cla17b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla17c":         print("cla17c", subtree.leaves()),review_lst.append(subtree.leaves())        
        if subtree.label() =="cla18":         print("cla18", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla18a":         print("cla18a", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla19":         print("cla19", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla19b":         print("cla19b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla20":         print("cla20", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla21":         print("cla21", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla22":         print("cla22", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla23":         print("cla23", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla24":         print("cla24", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla24b":         print("cla24b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla25":         print("cla25", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla25b":         print("cla25b", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla26":         print("cla26", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla27":         print("cla27", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla28":         print("cla28", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla29":         print("cla29", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla30":         print("cla30", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla31":         print("cla31", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla32":         print("cla32", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla33":         print("cla33", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla34":         print("cla34", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla35":         print("cla35", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla36":         print("cla36", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla37":         print("cla37", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla38":         print("cla38", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla39":         print("cla39", subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla40":       print("cla40",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla41":       print("cla41",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla42":       print("cla42",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla43":       print("cla43",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla44":       print("cla44",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla44b":       print("cla44b",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla45":       print("cla45",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla46":       print("cla46",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla47":       print("cla47",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla48":       print("cla48",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla49":       print("cla49",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla49b":       print("cla49b",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla50":       print("cla50",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla51":       print("cla51",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla52":       print("cla52",subtree.leaves()),review_lst.append(subtree.leaves())
        if subtree.label() =="cla53":       print("cla53",subtree.leaves()),review_lst.append(subtree.leaves())
#tree

NP [('Very', 'RB'), ('great', 'JJ'), ('product', 'NN')]
JN [('Very', 'RB'), ('great', 'JJ')]
cla31 [('money', 'NN'), ('!', '.')]
cla53 [('daughter', 'NN'), ('stole', 'VBD'), ('it', 'PRP')]
cla31 [('daughter', 'NN')]
cla33 [('stole', 'VBD'), ('it', 'PRP')]
cla19 [('stole', 'VBD'), ('it', 'PRP')]
cla33 [('ate', 'VB'), ('it', 'PRP'), (',', ',')]
cla19 [('ate', 'VB'), ('it', 'PRP'), (',', ',')]
cla22 [('guess', 'VBP'), ('it', 'PRP'), ("'s", 'VBZ'), ('a', 'DT'), ('good', 'JJ'), ('thing', 'NN'), ('I', 'PRP')]
cla2a [('it', 'PRP'), ("'s", 'VBZ'), ('a', 'DT'), ('good', 'JJ'), ('thing', 'NN')]
NP [('a', 'DT'), ('good', 'JJ'), ('thing', 'NN')]
JN [('good', 'JJ')]
NP [("n't", 'RB'), ('stand', 'VB'), ('oatmeal', 'NN'), ('.', '.')]
JN [("n't", 'RB')]
cla16 [('This', 'DT'), ('is', 'VBZ'), ('good', 'JJ')]
JN [('good', 'JJ')]
cla2a [('treats', 'NNS'), ('for', 'IN'), ('children', 'NNS')]
cla31 [('lunches', 'NNS'), ('though', 'IN'), ('or', 'CC')]
cla33 [('have', 'VB'), ('them', 'PRP')]
cla19 [('have', '

cla2a [('I', 'PRP'), ('recieved', 'VBD'), ('this', 'DT'), ('product', 'NN')]
cla33 [('Not', 'RB'), ('only', 'RB'), ('are', 'VBP'), ('they', 'PRP'), ('healthy', 'JJ')]
cla19 [('Not', 'RB'), ('only', 'RB'), ('are', 'VBP'), ('they', 'PRP')]
JN [('Not', 'RB'), ('only', 'RB')]
JN [('healthy', 'JJ')]
cla15 [('but', 'CC'), ('they', 'PRP'), ('taste', 'VBP'), ('great', 'JJ')]
JN [('great', 'JJ')]
cla40 [('are', 'VBP'), ('so', 'RB'), ('soft', 'JJ'), ('!', '.')]
JN [('so', 'RB'), ('soft', 'JJ'), ('!', '.')]
cla2a [('A', 'DT'), ('lot', 'NN'), ('of', 'IN'), ('store', 'NN'), ('bought', 'VBD'), ('oatmeal', 'JJ'), ('cookies', 'NNS')]
NP [('bought', 'VBD'), ('oatmeal', 'JJ'), ('cookies', 'NNS')]
JN [('oatmeal', 'JJ')]
cla21 [("n't", 'RB'), ('taste', 'VB')]
JN [("n't", 'RB')]
JN [('good', 'JJ')]
cla16 [('but', 'CC'), ('this', 'DT'), ('was', 'VBD'), ('so', 'RB'), ('good', 'JJ'), ('.', '.')]
JN [('so', 'RB'), ('good', 'JJ'), ('.', '.')]
cla21 [('I', 'PRP'), ('will', 'MD'), ('definitely', 'RB'), ('be', 'VB

cla31 [('Thanks', 'NNS')]
cla2a [('Influenster', 'NNP'), ('I', 'PRP'), ('got', 'VBD'), ('the', 'DT'), ('chance', 'NN')]
cla33 [('try', 'VB'), ('these', 'DT'), ('yummy', 'JJ'), ('Quaker', 'NNP'), ('soft', 'JJ'), ('baked', 'VBD'), ('raisin', 'JJ'), ('cookies', 'NNS'), ('!', '.')]
cla19 [('try', 'VB'), ('these', 'DT'), ('yummy', 'JJ'), ('Quaker', 'NNP'), ('soft', 'JJ'), ('baked', 'VBD'), ('raisin', 'JJ'), ('cookies', 'NNS'), ('!', '.')]
NP [('these', 'DT'), ('yummy', 'JJ'), ('Quaker', 'NNP')]
JN [('yummy', 'JJ')]
NP [('soft', 'JJ'), ('baked', 'VBD'), ('raisin', 'JJ'), ('cookies', 'NNS'), ('!', '.')]
JN [('soft', 'JJ')]
JN [('raisin', 'JJ')]
cla33 [('I', 'PRP'), ('will', 'MD'), ('be', 'VB'), ('buying', 'VBG'), ('a', 'DT'), ('box', 'NN')]
cla19 [('I', 'PRP'), ('will', 'MD'), ('be', 'VB'), ('buying', 'VBG'), ('a', 'DT'), ('box', 'NN')]
cla36 [('they', 'PRP'), ('are', 'VBP'), ('a', 'DT'), ('great', 'JJ'), ('snack', 'NN'), ('item', 'NN'), ('for', 'IN'), ('the', 'DT'), ('kids', 'NNS'), ('.', '.

cla36 [('I', 'PRP'), ('think', 'VBP'), ('for', 'IN'), ('a', 'DT'), ('quick', 'JJ'), ('breakfast', 'NN'), ('great', 'JJ'), ('choice', 'NN'), (',', ','), ('not', 'RB'), ('so', 'RB'), ('much', 'JJ'), ('as', 'IN'), ('a', 'DT'), ('sweet', 'JJ'), ('though', 'NN'), ('.', '.')]
cla2a [('I', 'PRP'), ('think', 'VBP'), ('for', 'IN'), ('a', 'DT'), ('quick', 'JJ'), ('breakfast', 'NN'), ('great', 'JJ'), ('choice', 'NN'), (',', ','), ('not', 'RB'), ('so', 'RB'), ('much', 'JJ')]
NP [('a', 'DT'), ('quick', 'JJ'), ('breakfast', 'NN')]
JN [('quick', 'JJ')]
NP [('great', 'JJ'), ('choice', 'NN'), (',', ',')]
JN [('great', 'JJ')]
JN [('not', 'RB'), ('so', 'RB'), ('much', 'JJ')]
NP [('a', 'DT'), ('sweet', 'JJ'), ('though', 'NN'), ('.', '.')]
JN [('sweet', 'JJ')]
cla16b [('the', 'DT'), ('oven', 'JJ'), ('fresh', 'JJ'), (',', ','), ('and', 'CC')]
cla16 [('the', 'DT'), ('oven', 'JJ'), ('fresh', 'JJ'), (',', ',')]
JN [('oven', 'JJ'), ('fresh', 'JJ'), (',', ',')]
JN [('full', 'JJ')]
cla31 [('flavor', 'NN'), ('!', 

cla1b [('These', 'DT'), ('cookies', 'NNS'), ('are', 'VBP'), ('so', 'RB'), ('soft', 'JJ'), ('and', 'CC'), ('yummy', 'NN'), ('.', '.')]
cla1a [('These', 'DT'), ('cookies', 'NNS'), ('are', 'VBP'), ('so', 'RB'), ('soft', 'JJ')]
JN [('so', 'RB'), ('soft', 'JJ')]
cla22 [('satisfied', 'VBD'), ('my', 'PRP$'), ('craving', 'NN'), ('for', 'IN'), ('a', 'DT'), ('sweet', 'NN')]
cla2a [('my', 'PRP$'), ('craving', 'NN'), ('for', 'IN'), ('a', 'DT'), ('sweet', 'NN')]
NP [('too', 'RB'), ('many', 'JJ'), ('calories', 'NNS'), ('.', '.')]
JN [('too', 'RB'), ('many', 'JJ')]
cla31 [('Soft', 'NNP'), (',', ','), ('and', 'CC')]
JN [('so', 'RB'), ('delicious', 'JJ'), ('!', '.')]
JN [('Very', 'RB'), ('tasty', 'JJ'), (',', ',')]
cla36 [('I', 'PRP'), ('loved', 'VBD'), ('the', 'DT'), ('softness', 'NN'), ('of', 'IN'), ('this', 'DT'), ('cookie', 'NN'), ('but', 'CC')]
cla2a [('I', 'PRP'), ('loved', 'VBD'), ('the', 'DT'), ('softness', 'NN')]
cla31 [('cookie', 'NN'), ('but', 'CC')]
cla21 [('not', 'RB'), ('to', 'TO'), ('kee

...

cla2b [('They', 'PRP'), ('are', 'VBP'), ('a', 'DT'), ('bit', 'NN'), ('crumbly', 'RB'), (',', ','), ('but', 'CC'), ('well', 'RB')]
cla2a [('They', 'PRP'), ('are', 'VBP'), ('a', 'DT'), ('bit', 'NN'), ('crumbly', 'RB'), (',', ',')]
JN [('crumbly', 'RB'), (',', ',')]
JN [('well', 'RB')]
cla33 [('see', 'VB'), ('their', 'PRP$'), ('smiling', 'NN')]
cla19 [('see', 'VB'), ('their', 'PRP$'), ('smiling', 'NN')]
cla1a [('The', 'DT'), ('Quaker', 'NNP'), ('Soft', 'NNP'), ('Baked', 'NNP'), ('Oatmeal', 'NNP'), ('cookies', 'NNS'), ('were', 'VBD'), ('delicious', 'JJ'), ('!', '.')]
JN [('delicious', 'JJ'), ('!', '.')]
JN [('soft', 'JJ'), (',', ',')]
JN [('fresh', 'JJ'), ('!', '.')]
cla2a [('Oatmeal', 'NNP'), ('raisin', 'VBZ'), ('an', 'DT'), ('all', 'DT'), ('time', 'NN')]
JN [('Not', 'RB'), ('overly', 'RB'), ('huge', 'JJ'), (',', ',')]
JN [('not', 'RB'), ('too', 'RB'), ('small', 'JJ'), ('either', 'RB'), ('.', '.')]
cla15 [('I', 'PRP'), ('have', 'VBP'), ('already', 'RB'), ('repurchased', 'VBN')]
JN [('alre

JN [('really', 'RB')]
cla2a [('this', 'DT'), ('Quaker', 'NNP'), ('Soft', 'NNP'), ('Baked', 'NNP'), ('Cookie', 'NNP'), ('in', 'IN'), ('the', 'DT'), ('Oatmeal', 'NNP'), ('raisin', 'NN'), ('flavor', 'NN')]
cla18 [('one', 'CD'), ('of', 'IN')]
NP [('my', 'PRP$'), ('favorite', 'JJ'), ('flavors', 'NNS'), ('.', '.')]
JN [('favorite', 'JJ')]
cla15 [('I', 'PRP'), ('do', 'VBP'), ("n't", 'RB'), ('feel', 'VB')]
JN [("n't", 'RB')]
cla31 [('baking', 'NN'), (',', ',')]
cla15c [('or', 'CC'), ('it', 'PRP'), ("'s", 'VBZ'), ('too', 'RB'), ('hot', 'JJ'), ('to', 'TO'), ('do', 'VB')]
cla15 [('or', 'CC'), ('it', 'PRP'), ("'s", 'VBZ'), ('too', 'RB'), ('hot', 'JJ')]
JN [('too', 'RB'), ('hot', 'JJ')]
JN [('so', 'RB'), (',', ',')]
cla31 [('share', 'NN')]
cla18 [('2', 'CD')]
cla3 [('year', 'NN'), ('old', 'JJ'), ('daughter', 'NN'), ('since', 'IN')]
NP [('old', 'JJ'), ('daughter', 'NN')]
JN [('old', 'JJ')]
NP [("n't", 'RB'), ('eat', 'VB'), ('cookies', 'NNS')]
JN [("n't", 'RB')]
cla2a [('front', 'NN'), ('of', 'IN'), 

cla47 [('How', 'WRB'), ('awesome', 'JJ'), ('is', 'VBZ')]
JN [('awesome', 'JJ')]
cla15 [('It', 'PRP'), ('was', 'VBD'), ('soft', 'JJ'), (',', ',')]
JN [('soft', 'JJ'), (',', ',')]
cla40 [('was', 'VBD'), ("n't", 'RB')]
JN [("n't", 'RB')]
cla31 [('taste', 'NN')]
cla31 [('cookie..', 'NN'), ('Just', 'NNP')]
NP [('the', 'DT'), ('yummy', 'JJ'), ('goodness..', 'NN')]
JN [('yummy', 'JJ')]
cla2b [('We', 'PRP'), ('tried', 'VBD'), ('the', 'DT'), ('Oatmeal', 'NNP'), ('raisin', 'NN'), ('cookie', 'NN'), ('and', 'CC'), ('the', 'DT'), ('raisin', 'NN')]
cla2a [('We', 'PRP'), ('tried', 'VBD'), ('the', 'DT'), ('Oatmeal', 'NNP'), ('raisin', 'NN'), ('cookie', 'NN')]
cla40 [('were', 'VBD'), ('good', 'JJ'), (',', ',')]
JN [('good', 'JJ'), (',', ',')]
cla15 [('they', 'PRP'), ('were', 'VBD'), ('soft', 'JJ'), ('not', 'RB'), ('hard..', 'VB')]
JN [('soft', 'JJ'), ('not', 'RB')]
cla53 [('daughter', 'NN'), ('asked', 'VBD'), ('me', 'PRP')]
cla31 [('daughter', 'NN')]
cla33 [('asked', 'VBD'), ('me', 'PRP')]
cla19 [('ask

cla3 [('This', 'DT'), ('product', 'NN'), ('gives', 'VBZ'), ('Quaker', 'NNP'), ('a', 'DT'), ('bad', 'JJ'), ('name', 'NN'), ('.', '.')]
NP [('gives', 'VBZ'), ('Quaker', 'NNP')]
NP [('a', 'DT'), ('bad', 'JJ'), ('name', 'NN'), ('.', '.')]
JN [('bad', 'JJ')]
JN [('So', 'RB')]
cla12 [("'m", 'VBP'), ('searching', 'VBG'), ('for', 'IN'), ('her', 'PRP'), (',', ',')]
cla41 [('I', 'PRP'), ('find', 'VBP'), ('her', 'PRP'), ('in', 'IN'), ('my', 'PRP$'), ('bathroom', 'NN'), ('shower', 'NN'), ('with', 'IN'), ('the', 'DT'), ('cookie', 'NN'), ('almost', 'RB')]
cla33 [('I', 'PRP'), ('find', 'VBP'), ('her', 'PRP')]
cla19 [('I', 'PRP'), ('find', 'VBP'), ('her', 'PRP')]
cla2a [('my', 'PRP$'), ('bathroom', 'NN'), ('shower', 'NN'), ('with', 'IN'), ('the', 'DT'), ('cookie', 'NN'), ('almost', 'RB')]
JN [('almost', 'RB')]
cla1a [('This', 'DT'), ('cookie', 'NN'), ('is', 'VBZ'), ('full', 'JJ')]
JN [('full', 'JJ')]
cla31 [('flavor', 'NN'), ('and', 'CC'), ('at', 'IN')]
NP [('a', 'DT'), ('great', 'JJ'), ('price', 'NN'

cla15 [('I', 'PRP'), ('was', 'VBD'), ('going', 'VBG')]
cla31 [('products', 'NNS')]
cla14 [('sent', 'VBN'), ('to', 'TO'), ('test', 'VB')]
cla2a [('I', 'PRP'), ('opened', 'VBD'), ('the', 'DT'), ('Quaker', 'NNP'), ('Soft', 'NNP'), ('Baked', 'NNP'), ('Oatmeal', 'NNP'), ('cookie', 'NN')]
cla33 [('ate', 'VB'), ('it', 'PRP'), ('.', '.')]
cla19 [('ate', 'VB'), ('it', 'PRP'), ('.', '.')]
JN [('Soft', 'JJ')]
cla31 [('Chewy', 'NNP'), (',', ',')]
cla31 [('son', 'NN'), ('and', 'CC')]
cla2a [('I', 'PRP'), ('shared', 'VBD'), ('a', 'DT'), ('free', 'JJ'), ('sample', 'NN')]
NP [('a', 'DT'), ('free', 'JJ'), ('sample', 'NN')]
JN [('free', 'JJ')]
cla2a [('the', 'DT'), ('Quaker', 'NNP'), ('Soft', 'NNP'), ('Baked', 'NNP'), ('Oatmeal', 'NNP'), ('Cookies', 'NNPS'), ('from', 'IN'), ('influenster', 'NN'), ('.', '.')]
cla15 [('They', 'PRP'), ('are', 'VBP'), ('soft', 'JJ')]
JN [('soft', 'JJ')]
cla31 [('chewy', 'NN'), ('.', '.')]
cla15b [('They', 'PRP'), ('are', 'VBP'), ('delicious', 'JJ'), ('and', 'CC'), ('not', '

In [33]:
sent_lst2[0:5]

[['I', 'love', 'these', 'cookies', '!'],
 ['Not',
  'only',
  'are',
  'they',
  'healthy',
  'but',
  'they',
  'taste',
  'great',
  'and',
  'are',
  'so',
  'soft',
  '!'],
 ['I',
  'will',
  'definitely',
  'add',
  'these',
  'to',
  'my',
  'grocery',
  'list',
  '!'],
 ['Quaker',
  'Soft',
  'Baked',
  'Oatmeal',
  'Cookies',
  'with',
  'raisins',
  'are',
  'a',
  'delicious',
  'treat',
  ',',
  'great',
  'for',
  'anytime',
  'of',
  'day',
  '.'],
 ['For',
  'example',
  ':',
  '<',
  'br',
  '/',
  '>',
  '<',
  'br',
  '/',
  '>',
  '--',
  'at',
  'breakfast',
  ',',
  'I',
  'had',
  'one',
  'with',
  'a',
  'large',
  'banana',
  'and',
  'a',
  'cup',
  'of',
  'coffee',
  ',',
  'and',
  'felt',
  'I',
  "'d",
  'had',
  'a',
  'relatively',
  '``',
  'healthy',
  "''",
  'start',
  'to',
  'the',
  'day.',
  '<',
  'br',
  '/',
  '>',
  '<',
  'br',
  '/',
  '>',
  '--',
  'the',
  'next',
  'day',
  'at',
  'lunch',
  ',',
  'following',
  'a',
  'tuna',
  'sand

In [9]:
for line in review_lst:
    if len(line) >=3:
        print(line)

[('Very', 'RB'), ('great', 'JJ'), ('product', 'NN')]
[('daughter', 'NN'), ('stole', 'VBD'), ('it', 'PRP')]
[('ate', 'VB'), ('it', 'PRP'), (',', ',')]
[('ate', 'VB'), ('it', 'PRP'), (',', ',')]
[('guess', 'VBP'), ('it', 'PRP'), ("'s", 'VBZ'), ('a', 'DT'), ('good', 'JJ'), ('thing', 'NN'), ('I', 'PRP')]
[('it', 'PRP'), ("'s", 'VBZ'), ('a', 'DT'), ('good', 'JJ'), ('thing', 'NN')]
[('a', 'DT'), ('good', 'JJ'), ('thing', 'NN')]
[("n't", 'RB'), ('stand', 'VB'), ('oatmeal', 'NN'), ('.', '.')]
[('This', 'DT'), ('is', 'VBZ'), ('good', 'JJ')]
[('treats', 'NNS'), ('for', 'IN'), ('children', 'NNS')]
[('lunches', 'NNS'), ('though', 'IN'), ('or', 'CC')]
[('go', 'VBP'), ('hungry.', 'JJ'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('Also', 'RB'), (',', ',')]
[('go', 'VBP'), ('hungry.', 'JJ'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP')]
[('and', 'CC'), ('it', 'PRP'), 

[('bite', 'NN'), ('and', 'CC'), ('pulls', 'NNS')]
[('you', 'PRP'), ('want', 'VB'), ('more', 'JJR'), ('.', '.')]
[('it', 'PRP'), ('is', 'VBZ'), ('not', 'RB'), ('particularly', 'RB'), ('healthy-', 'JJ'), ('not', 'RB'), ('bad', 'JJ'), (',', ',')]
[('not', 'RB'), ('particularly', 'RB'), ('healthy-', 'JJ'), ('not', 'RB'), ('bad', 'JJ'), (',', ',')]
[('but', 'CC'), ('the', 'DT'), ('stats', 'JJ'), ('arent', 'JJ'), ('great', 'JJ')]
[('stats', 'JJ'), ('arent', 'JJ'), ('great', 'JJ')]
[('small-ish', 'JJ'), ('cookie.', 'NN'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('Give', 'VB'), ('it', 'PRP')]
[('small-ish', 'JJ'), ('cookie.', 'NN'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('Give', 'VB')]
[('small-ish', 'JJ'), ('cookie.', 'NN'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP')]
[('try', 'NN')

[('it', 'PRP'), ('was', 'VBD'), ('delicious', 'JJ')]
[('daughter', 'NN'), ('enjoyed', 'VBD'), ('it', 'PRP'), ('as', 'RB'), ('well', 'RB'), (',', ','), ('i', 'JJ'), ('cant', 'VBP'), ('wait', 'NN')]
[('enjoyed', 'VBD'), ('it', 'PRP'), ('as', 'RB'), ('well', 'RB'), (',', ','), ('i', 'JJ'), ('cant', 'VBP'), ('wait', 'NN')]
[('enjoyed', 'VBD'), ('it', 'PRP'), ('as', 'RB'), ('well', 'RB'), (',', ','), ('i', 'JJ'), ('cant', 'VBP'), ('wait', 'NN')]
[('as', 'RB'), ('well', 'RB'), (',', ','), ('i', 'JJ'), ('cant', 'VBP'), ('wait', 'NN')]
[('as', 'RB'), ('well', 'RB'), (',', ',')]
[('get', 'VB'), ('my', 'PRP$'), ('hands', 'NNS')]
[('get', 'VB'), ('my', 'PRP$'), ('hands', 'NNS')]
[('I', 'PRP'), ('received', 'VBD'), ('a', 'DT'), ('Quaker', 'NNP'), ('Soft', 'NNP'), ('Baked', 'NNP'), ('Oatmeal', 'NNP'), ('cookie', 'NN')]
[('my', 'PRP$'), ('Mom', 'NNP'), ('Voxbox', 'NNP'), ('from', 'IN'), ('Influenster', 'NNP'), (',', ','), ('and', 'CC'), ('it', 'PRP'), ('was', 'VBD'), ('a', 'DT'), ('big', 'JJ'), ('hi

[('are', 'VBP'), ('a', 'DT'), ('few', 'JJ')]
[('I', 'PRP'), ('go', 'VBP'), ('digging', 'VBG'), ('for', 'IN'), ('a', 'DT'), ('few', 'JJ'), ('more', 'JJR'), ('cookies', 'NNS')]
[('a', 'DT'), ('few', 'JJ'), ('more', 'JJR'), ('cookies', 'NNS')]
[('conquer', 'VB'), ('my', 'PRP$'), ('sweet', 'JJ'), ('tooth', 'NNS')]
[('conquer', 'VB'), ('my', 'PRP$'), ('sweet', 'JJ'), ('tooth', 'NNS')]
[('my', 'PRP$'), ('sweet', 'JJ'), ('tooth', 'NNS')]
[('toddler', 'NN'), ('loves', 'VBZ'), ('them', 'PRP'), ('too', 'RB'), ('!', '.')]
[('loves', 'VBZ'), ('them', 'PRP'), ('too', 'RB'), ('!', '.')]
[('I', 'PRP'), ('received', 'VBD'), ('this', 'DT'), ('product', 'NN'), ('courtesy', 'NN'), ('of', 'IN'), ('Influenster', 'NNP'), ('and', 'CC')]
[('I', 'PRP'), ('received', 'VBD'), ('this', 'DT'), ('product', 'NN'), ('courtesy', 'NN')]
[('this', 'DT'), ('is', 'VBZ'), ('delicious..it', 'NN'), ('was', 'VBD'), ('soft', 'JJ')]
[('this', 'DT'), ('is', 'VBZ'), ('delicious..it', 'NN')]
[('These', 'DT'), ('cookies', 'NNS'), (

[("n't", 'RB'), ('have', 'VB'), ('made', 'VBN'), ('a', 'DT'), ('better', 'JJR'), ('cookie', 'NN'), ('!', '.')]
[('a', 'DT'), ('better', 'JJR'), ('cookie', 'NN'), ('!', '.')]
[('Soft', 'JJ'), ('Baked', 'NNP'), ('Oatmeal', 'NNP'), ('Raisin', 'NNP'), ('Cookies', 'NNP'), ('are', 'VBP'), ('indeed', 'RB'), ('soft', 'JJ')]
[('Soft', 'JJ'), ('Baked', 'NNP'), ('Oatmeal', 'NNP'), ('Raisin', 'NNP'), ('Cookies', 'NNP')]
[('I', 'PRP'), ('received', 'VBD'), ('a', 'DT'), ('sample', 'NN')]
[('the', 'DT'), ('Mom', 'NNP'), ('VoxBox', 'NNP'), ('via', 'IN'), ('Influenster', 'NNP'), ('.', '.')]
[('My', 'PRP$'), ('only', 'JJ'), ('problem', 'NN'), ('is', 'VBZ')]
[('My', 'PRP$'), ('only', 'JJ'), ('problem', 'NN')]
[('is', 'VBZ'), ('not', 'RB'), ('remotely', 'RB')]
[('they', 'PRP'), ('are', 'VBP'), ('individually', 'RB'), ('wrapped.', 'JJ'), ('<', 'NNP'), ('br', 'NN'), ('/', 'NNP'), ('>', 'NNP'), ('Specially', 'NNP'), ('since', 'IN')]
[('are', 'VBP'), ('individually', 'RB'), ('wrapped.', 'JJ'), ('<', 'NNP'), (

[('this', 'DT'), ('fine', 'JJ'), ('product', 'NN'), (',', ',')]
[('is', 'VBZ'), ('that', 'IN'), ('it', 'PRP')]
[('could', 'MD'), ('be', 'VB'), ('a', 'DT'), ('preferred', 'JJ'), ('snack', 'NN'), ('food', 'NN')]
[('could', 'MD'), ('be', 'VB'), ('a', 'DT'), ('preferred', 'JJ'), ('snack', 'NN'), ('food', 'NN')]
[('a', 'DT'), ('preferred', 'JJ'), ('snack', 'NN'), ('food', 'NN')]
[('have', 'VBP'), ('had', 'VBN'), ('multiple', 'JJ'), ('dental', 'JJ'), ('fillings', 'NNS'), (',', ',')]
[('dental', 'JJ'), ('damage', 'NN'), ('.', '.')]
[('were', 'VBD'), ('so', 'RB'), ('good', 'JJ'), (',', ','), ('My', 'NNP'), ('daughter', 'NN')]
[('so', 'RB'), ('good', 'JJ'), (',', ',')]
[('I', 'PRP'), ('shared', 'VBD'), ('the', 'DT'), ('first', 'JJ')]
[('got', 'VBD'), ('another', 'DT'), ('box', 'NN'), ('!', '.')]
[('got', 'VBD'), ('another', 'DT'), ('box', 'NN'), ('!', '.')]
[('I', 'PRP'), ('am', 'VBP'), ('pretty', 'RB'), ('picky', 'JJ')]
[('like', 'VBP'), ('homemade', 'JJ'), ('cookies', 'NNS'), ('.', '.')]
[('t

[('could', 'MD'), ('eat', 'VB'), ('a', 'DT'), ('whole', 'JJ'), ('box', 'NN'), ('!', '.')]
[('could', 'MD'), ('eat', 'VB'), ('a', 'DT'), ('whole', 'JJ'), ('box', 'NN'), ('!', '.')]
[('a', 'DT'), ('whole', 'JJ'), ('box', 'NN'), ('!', '.')]
[('are', 'VBP'), ('to', 'TO'), ('die', 'VB')]
[('It', 'PRP'), ('is', 'VBZ'), ('soft', 'JJ')]
[("n't", 'RB'), ('have', 'VB'), ('to', 'TO'), ('worry', 'VB')]
[('your', 'PRP$'), ('on', 'IN'), ('the', 'DT'), ('go', 'NN'), ('.', '.')]
[('definitely', 'RB'), ('be', 'VB'), ('purchasing', 'VBG'), ('this', 'DT'), ('item', 'NN'), ('more', 'RBR'), ('often', 'RB'), ('!', '.')]
[('definitely', 'RB'), ('be', 'VB'), ('purchasing', 'VBG'), ('this', 'DT'), ('item', 'NN')]
[('more', 'RBR'), ('often', 'RB'), ('!', '.')]
[('This', 'DT'), ('is', 'VBZ'), ('pretty', 'RB'), ('good', 'JJ')]
[('a', 'DT'), ('store', 'NN'), ('bought', 'VBD'), ('cookie', 'NN'), ('!', '.')]
[('bought', 'VBD'), ('cookie', 'NN'), ('!', '.')]
[('the', 'DT'), ('Mom', 'NNP'), ('VoxBox', 'NNP'), ('from',

In [ ]:
Extract

In [58]:
Ext_sen_df = pd.DataFrame(review_lst)
Ext_sen_df.to_csv("Ext_sen.csv", index=False, header=False)


### Functions to get the Wordnet Tag  and lemmatized words forms

In [10]:
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

def to_wntag(pos):
    """
    Given a Penn tag, returns the corresponding Wordnet pos tag.
    Note that WordNet only contain nouns/verbs/adjectives/adverbs.
    """
    ch = pos[0]
    if ch == 'N':
        return wn.NOUN #noun
    elif ch == 'V':
        return wn.VERB # verb
    elif ch == 'J':
        return wn.ADJ # adjective
    elif ch == 'R':
        return wn.ADV # adverb
    else:
        return None

def wn_lemmatize(token, wntag):
    """
    Applies WordNet morpher (which requires a pos) and 
    returns the morphological lemma form, or None.
    """
    return wn.morphy(token, wntag)

### Step-5: Parse segments to get the Synset[0] and Synset[ALL]

In [88]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
#df_lst = [0 for item in range(len(review_lst2))]
df_syn0_lst = []
df_syn_all_lst = []
df_tup_lst = []

cnt=0
for tups in review_lst[0:10]:
#for tups in test_lst:   
    if len(tups)<3:
        continue
    
    lst=[]
    tot_pscore_syn0=0
    tot_pscore_syn_all = 0
    word_cnt=0
    
    for i in range(len(tups)):
        #print("tups[i]",tups[i])
        tups[i]=list(tups[i]) #Convert the list back to tuples
        w=tups[i][0].lower() #Lower the case of words and assign the word from the first element of each tuple
        tag=tups[i][1] #Assign the tag from the 2nd element of tuples
        #print("w",w)
        #print("tag",tag)
        
        #If the word is a negative contraction suffix ('negcontractions'), 
        #change it to the word: "not"
        if w in negcontractions:
            w="not"

        # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
        wntag = to_wntag(tag) # in WordNet, either noun/verb/adj/adv
        if wntag is not None and (w not in stopwords or w in negwords): 
            print("1")
            tups[i][0] = w 
            word_cnt+=1
            lemma = wn_lemmatize(w, wntag) #lemmatize token

            if lemma is None:
                print("2")
                tups[i]=tuple(tups[i])
                continue

            synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word

            if not synsets:
                print("3")
                tups[i]=tuple(tups[i])
                continue

            # take the first sense
            synset = synsets[0]
            #print(synsets)
            #print(synset)
            swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
            #print(swn_synset)
            pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
            #print(pscore)
            tot_pscore_syn0+=pscore
            
            tot_pscore_syn=0
            if len(synsets)>1:
                for synset in synsets:
                    swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
                    #print(swn_synset)
                    pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
                    #print("pscore",pscore)
                    tot_pscore_syn += pscore
                tot_pscore_syn_all += (tot_pscore_syn/len(synsets))
                #print("tot_pscore_syn_all",tot_pscore_syn_all)
            else: tot_pscore_syn_all += tot_pscore_syn0

            
            
            
            #Convert the list back to tuples
            tups[i]=tuple(tups[i])

        else:
            tups[i][0] = w 
            tups[i]=tuple(tups[i])
            
    df_syn0_lst.append(tot_pscore_syn0/word_cnt)
    df_syn_all_lst.append(tot_pscore_syn_all/word_cnt)
    #print("Word count",word_cnt)
    df_tup_lst.append(tups)
    cnt+=1
    
    
#Concatnate the tuples and Polarity Score in a DataFrame
#df = pd.DataFrame({ i:pd.Series(value) for i, value in enumerate(df_lst) })
#df1= pd.DataFrame(df_lst1, columns= ["Polarity_Score"] )
#df2= pd.DataFrame(df_lst2)
#df = pd.concat((df1,df2), axis=1)
#df = df.sort_values(by=["Polarity_Score",0,1,2,3,4,5,6,7,8,9,10], 

#                   ascending=[False, True, True,True, True,True,True, True,True, True,True,True])

1
1
1
1
1
1
1
1
2
1
1


In [203]:
test_lst= []

sent = "NO comemnt"
sent = word_tokenize(sent)
#print(sent)
tree= cp.parse(nltk.pos_tag(sent))
cnt=0
for subtree in tree.subtrees():
    print(cnt)
    #print(subtree.leaves())
    test_lst.append(subtree.leaves())
    cnt+=1
test_lst[0]

0
1


[('NO', 'DT'), ('comemnt', 'NN')]

In [259]:
Counter(df_tup_lst[0]).most_common()[0][]

1

In [260]:
PRP_cnt_lst=[]
no_cnt_lst=[]
never_cnt_lst=[]
quite_cnt_lst=[]
quite_cnt_lst=[]
but_cnt_lst=[]
very_cnt_lst=[]
more_cnt_lst=[]
br_cnt_lst=[]
else_cnt_lst=[]
must_cnt_lst=[]
many_cnt_lst=[]
up_cnt_lst=[]
so_cnt_lst=[]
all_cnt_lst=[]
dots_cnt_lst=[]
er_cnt_lst=[]
est_cnt_lst=[]
ous_cnt_lst=[]
ly_cnt_lst=[]
exclamation_mark_cnt_lst=[]
question_mark_cnt_lst=[]
repeat_word_cnt_lst=[]

pre_item=()
new_item=()

for tups in df_tup_lst[0:5]:
#for tups in test_lst[0]:
    print(tups)
    PRP_cnt , no_cnt, very_cnt, more_cnt, br_cnt, else_cnt,must_cnt, many_cnt,but_cnt,up_cnt,so_cnt,all_cnt,dots_cnt,never_cnt,quite_cnt,not_cnt= 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
    exclamation_mark_cnt, question_mark_cnt = 0,0
    er_cnt,est_cnt,ous_cnt,ly_cnt = 0,0,0,0
    repeat_word_cnt = 0
    
    for item in tups:
        pre_item=item
        
        #Check consecutive repeated word 
        if new_item == pre_item:
            repeat_word_cnt+=1
        else:
            new_item = pre_item
        
        if item[0].lower() == "no":
            print("no")
            no_cnt+=1
            
        if item[0].lower() == "never":
            print("never")
            never_cnt+=1
            
        if item[0].lower() == "quite":
            print("quite")
            quite_cnt+=1
        
        if item[0].lower() == "but":
            print("but")
            but_cnt+=1
    
        if item[0].lower() == "very":
            print("very")
            very_cnt+=1
        
        if item[0].lower() == "more":
            print("more")
            more_cnt+=1
                
        if item[0].lower() == "br":
            print("br")
            br_cnt+=1
        
        if item[0].lower() == "else":
            print("else")
            else_cnt+=1
        if item[0].lower() == "must":
            print("must")
            must_cnt+=1
        if item[0].lower() == "many":
            print("many")
            many_cnt+=1
                
        if item[0].lower() == "up":
            print("up")
            up_cnt+=1
        if item[0].lower() == "so":
            print("so")
            so_cnt+=1
        if item[0].lower() == "all":
            print("all")
            all_cnt+=1
        if item[0] == "...":
            print("...")
            dots_cnt+=1
        if item[0] == "!":
            print("!")
            exclamation_mark_cnt+=1
        
        if item[0] == "?":
            print("?")
            question_mark_cnt+=1
        if item[0] == "not":
            print("not")
            not_cnt+=1
            
            
            
            
        
        if (item[0][-2:] == "er") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("er")
            er_cnt+=1
             
        if (item[0][-3:] == "est") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("est")
            est_cnt+=1
                
        if (item[0][-4:] == "ous") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("ous")
            ous_cnt+=1
                
        if (item[0][-2:] == "ly") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("ly")
            ly_cnt+=1
                
        if (item[0][-2:] == "dy") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("dy")
            dy_cnt+=1
        
        if (item[0][-2:] == "ful") and (item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("ful")
            ful_cnt+=1
            
        if (item[0][-2:] == "ed") and ( item[1]=="VBD" or item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("ed")
            ed_cnt+=1
        
        if (item[0][-4:] == "less") and ( item[1]=="JJ" or item[1]=="JJR" or item[1]=="JJS" or item[1]=="RB" or item[1]=="RBR" or item[1]=="RBS"):
            print("less")
            less_cnt+=1
                
                
        if item[1] =="CC":
            print("CC")
            CC_cnt+=1
        if item[1] =="CD":
            print("CD")
            CD_cnt+=1
        if item[1] =="DT":
            print("DT")
            DT_cnt+=1
        if item[1] =="IN":
            print("IN")
            IN_cnt+=1
        if item[1] =="JJ":
            print("JJ")
            JJ_cnt+=1
        if item[1] =="JJR":
            print("JJR")
            JJR_cnt+=1
        if item[1] =="JJS":
            print("JJS")
            JJS_cnt+=1
        if item[1] =="MD":
            print("MD")
            MD_cnt+=1
        if item[1] =="NN":
            print("NN")
            NN_cnt+=1
        if item[1] =="NNS":
            print("NNS")
            NNS_cnt+=1
        if item[1] =="NNP":
            print("NNP")
            NNP_cnt+=1
        if item[1] =="NNPS":
            print("NNPS")
            NNPS_cnt+=1
        if item[1] =="POS":
            print("POS")
            POS_cnt+=1
        if item[1] =="PDT":
            print("PDT")
            PDT_cnt+=1
        if item[1] =="PRP":
            print("PRP")
            PRP_cnt+=1
        if item[1] =="PRP$":
            print("PRP$")
            PRP_dol_cnt+=1
        if item[1] =="RB":
            print("RB")
            RB_cnt+=1
        if item[1] =="RBR":
            print("RBR")
            RBR_cnt+=1
        if item[1] =="RBS":
            print("RBS")
            RBS_cnt+=1
        if item[1] =="TO":
            print("TO")
            TO_cnt+=1
        if item[1] =="VB":
            print("VB")
            VB_cnt+=1
        if item[1] =="VBD":
            print("VBD")
            VBD_cnt+=1
        if item[1] =="VBG":
            print("VBG")
            VBG_cnt+=1
        if item[1] =="VBN":
            print("VBN")
            VBN_cnt+=1
        if item[1] =="WDT":
            print("WDT")
            WDT_cnt+=1
        if item[1] =="WP":
            print("WP")
            WP_cnt+=1
        if item[1] =="WP$":
            print("WP$")
            WP_dol_cnt+=1
        if item[1] =="WRB":
            print("WRB")
            WRB_cnt+=1
        if item[1] ==".":
            print(".")
            dot_cnt+=1

        
    PRP_cnt_lst.append(PRP_cnt)
    CC_cnt_lst.append(CC_cnt)
    CD_cnt_lst.append(CD_cnt)
    
    
    repeat_word_cnt_lst.append(repeat_word_cnt)
    no_cnt_lst.append(no_cnt)
    never_cnt_lst.append(never_cnt)
    quite_cnt_lst.append(quite_cnt)
    but_cnt_lst.append(but_cnt)
    very_cnt_lst.append(very_cnt)
    more_cnt_lst.append(more_cnt)
    br_cnt_lst.append(br_cnt)
    else_cnt_lst.append(else_cnt)
    must_cnt_lst.append(must_cnt)
    many_cnt_lst.append(many_cnt)
    up_cnt_lst.append(up_cnt)
    so_cnt_lst.append(so_cnt)
    all_cnt_lst.append(all_cnt)
    dots_cnt_lst.append(dots_cnt)
    er_cnt_lst.append(er_cnt)
    est_cnt_lst.append(est_cnt)
    ous_cnt_lst.append(ious_cnt)
    ly_cnt_lst.append(ly_cnt)
    dy_cnt_lst.append(dy_cnt)
    ful_cnt_lst.append(ful_cnt)
    less_cnt_lst.append(less_cnt)
    exclamation_mark_cnt_lst.append(exclamation_mark_cnt)
    
    
print("no",no_cnt_lst)
print("never",never_cnt_lst)
print("PRP",PRP_cnt_lst)
print("er",er_cnt_lst)


SyntaxError: invalid syntax (<ipython-input-260-0b527b83d2d8>, line 101)

In [91]:
dog = wn.synset('dog.n.01')
cat = wn.synset('cat.n.01')
dog.path_similarity(cat)

0.2

In [161]:
syn_good = wn.synsets("good", pos=wntag)[0]
syn_declicious = wn.synsets("delicious")[0]
syn_excellent = wn.synsets("excellent")[0]
syn_great = wn.synsets("great", pos=wntag)[0]

syn_w = wn.synsets("bad", pos=wntag)[0]
#syn_w = wn.synsets("delicious", pos=wntag)[0]

syn_good.path_similarity(syn_great)
#syn_excellent
#syn_great
#syn_good

syn_good_n = wn.synsets("good", pos=wntag)[0]
syn_good_a = wn.synsets("good", pos=wntag)[4]
syn_good_s = wn.synsets("good", pos=wntag)[10]

wn.synsets("delicious")

#wn.synsets("excellent", pos=wntag)
wn.synsets("excellent")

[Synset('excellent.s.01')]

In [63]:

import nltk
stopwords = nltk.corpus.stopwords.words('english')
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
label_lst = []
avg_label_score_lst = []


cp = nltk.RegexpParser(grammar)


start=time.time()
token_line_lst = []
cnt=0

sents=["I love this food. I really like it"]

for line in sents:
    token_line_lst=[]
    print(line)
    for token_line in sent_tokenize(line):
        print(token_line)
        token_line_lst.append(token_line)
    
    sentlist = [word_tokenize(line) for line in token_line_lst]
    print(sentlist)
    
    review_lst=[]    
    for sent in sentlist: #Iterate the Review sentlist 
        tree = cp.parse(nltk.pos_tag(sent)) #pos_tag the sentence and parse the sentence
        #print(tree)
        for subtree in tree.subtrees():
            if subtree.label() =="NP":
                #print("NP")
                #print(subtree.leaves())    
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause1a":
                #print("clause1a")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause1b":
                #print("clause1b")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause2a":
                #print("clause2a")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause2b":
                #print("clause2b")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause3":
                #print("clause3")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause4":
                #print("clause4")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
            if subtree.label() =="clause5":
                #print("clause5")
                #print(subtree.leaves())
                review_lst.append(subtree.leaves())
        
    #Filtering out all the segments which do not have feature words              
    review_lst2=[] #initialize a new list
    for line in review_lst: #iterating the list with extracted segments
        for item in line:
            if item[0] in feature_lst: #Feature list is obtained in Step 3
                review_lst2.append(line) #Append all the segments have feature words in the new list
                continue
        
    

##################
    #print(len(review_lst))
    #print(review_lst)
    #print("# of segments",len(review_lst2))
    #print(review_lst2)
##################
    df_lst1=[]
    df_lst2=[]

    for tups in review_lst2:
        lst=[]
        tot_pscore=0
        word_cnt=0
        
        for i in range(len(tups)):
            tups[i]=list(tups[i]) #Convert the list back to tuples
            w=tups[i][0].lower() #Lower the case of words and assign the word from the first element of each tuple
            tag=tups[i][1] #Assign the tag from the 2nd element of tuples

            #If the word is a negative contraction suffix ('negcontractions'), 
            #change it to the word: "not"
            if w in negcontractions:
                w="not"
                
            # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
            wntag = to_wntag(tag) # in WordNet, either noun/verb/adj/adv
            if wntag is not None and (w not in stopwords or w in negwords): 
                #print(w)
                tups[i][0] = w 
                word_cnt+=1
                lemma = wn_lemmatize(w, wntag) #lemmatize token

                if lemma is None:
                    tups[i]=tuple(tups[i])
                    continue

                synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word

                if not synsets:
                    tups[i]=tuple(tups[i])
                    continue

                # take the first sense
                synset = synsets[0]
                swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
                pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
                tot_pscore+=pscore

                #Convert the list back to tuples
                tups[i]=tuple(tups[i])

            else:
                tups[i][0] = w 
                tups[i]=tuple(tups[i])

        if word_cnt == 0: df_lst1.append(0)
        else: df_lst1.append(tot_pscore/word_cnt)
        #print("df_lst1:",df_lst1)


        df_lst2.append(tups)

    
    if np.mean(df_lst1) > 0 :
        label=1
    else:
        label=0
    #print(df_lst2)
    avg_label_score_lst.append(np.mean(df_lst1))
    label_lst.append(label)
   
    print(cnt)
    cnt+=1
end=time.time()
print("%d, Time taken %f min"%(cnt+1, ((end-start)/ 60)))
#print("avg",avg_label_score_lst)    
#print("label",label_lst)
print(len(label_lst))

I love this food. I really like it
I love this food.
I really like it
[['I', 'love', 'this', 'food', '.'], ['I', 'really', 'like', 'it']]
0
2, Time taken 0.000164 min
1


/Users/KevQuant/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/KevQuant/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [119]:
avg_label_score_lst

NameError: name 'avg_label_score_lst' is not defined

In [131]:
y__true = df.Score.copy()
#y__pred = label_lst
y__true[y__true < 3] =0
y__true[y__true > 3] =1


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y__true,y__pred,labels=[0,1,3])
cm
#(cm[i,i] / sum(cm[i]))

array([[ 52429,  29608,      0],
       [223949, 219828,      0],
       [ 22287,  20353,      0]])

In [130]:
Counter(y__true)




Counter({0: 82037, 1: 443777, 3: 42640})

In [116]:
len(label_lst)
df.Score.head(5)

0    5
1    1
2    4
3    2
4    5
Name: Score, dtype: int64

In [105]:
df_pred = df.copy()
df_pred.shape

(568454, 10)

In [112]:
pred_col = pd.DataFrame(label_lst, columns=["pred"])
#pd.concat((df_pred, pred_col))
pred_col

,pred
0,1
1,0
2,1
3,0
4,0
5,1
6,0
7,1
8,0
9,1


### Step-6: Polarity Score

##### Given Functions

In [ ]:
w="comfortable"
tag="JJ"
wntag = to_wntag(tag)
lemma = wn_lemmatize(w, wntag)
synsets = wn.synsets(lemma, pos=wntag)
print(synsets)
synset = synsets[2]


swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
print(swn_synset)
pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
pscore

In [ ]:
import nltk
stopwords = nltk.corpus.stopwords.words('english')
negcontractions = ["n't", "'t"]
negwords = ['not', 'no', "nor"]
df_lst = [0 for item in range(len(review_lst2))]
df_lst2 = []
df_lst1 = []

cnt=0
for tups in review_lst2:
    lst=[]
    tot_pscore=0
    word_cnt=0
    for i in range(len(tups)):
        tups[i]=list(tups[i]) #Convert the list back to tuples
        w=tups[i][0].lower() #Lower the case of words and assign the word from the first element of each tuple
        tag=tups[i][1] #Assign the tag from the 2nd element of tuples
        
        #If the word is a negative contraction suffix ('negcontractions'), 
        #change it to the word: "not"
        if w in negcontractions:
            w="not"

        # a word is a tuple (w, tag) such as ('bathrooms', 'NNS')
        wntag = to_wntag(tag) # in WordNet, either noun/verb/adj/adv
        if wntag is not None and (w not in stopwords or w in negwords): 
            #print(w)
            tups[i][0] = w 
            word_cnt+=1
            lemma = wn_lemmatize(w, wntag) #lemmatize token

            if lemma is None:
                tups[i]=tuple(tups[i])
                continue

            synsets = wn.synsets(lemma, pos=wntag) #all synsets of the word

            if not synsets:
                tups[i]=tuple(tups[i])
                continue

            # take the first sense
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name()) #SentiWordNet synset
            pscore = swn_synset.pos_score() - swn_synset.neg_score() #polarity score for the word
            tot_pscore+=pscore
            
            #Convert the list back to tuples
            tups[i]=tuple(tups[i])

        else:
            tups[i][0] = w 
            tups[i]=tuple(tups[i])
            
    df_lst1.append(tot_pscore/word_cnt)
    df_lst2.append(tups)
    cnt+=1
    
    
#Concatnate the tuples and Polarity Score in a DataFrame
#df = pd.DataFrame({ i:pd.Series(value) for i, value in enumerate(df_lst) })
df1= pd.DataFrame(df_lst1, columns= ["Polarity_Score"] )
df2= pd.DataFrame(df_lst2)
df = pd.concat((df1,df2), axis=1)
df = df.sort_values(by=["Polarity_Score",0,1,2,3,4,5,6,7,8,9,10], 
                    ascending=[False, True, True,True, True,True,True, True,True, True,True,True])

In [ ]:
len(review_lst2)

In [ ]:
pre_lst=[]
new_lst=[]
cnt=0
with open("XXXX-rooms.txt", "w") as f:
    for row in range(0,df.shape[0]):
        new_lst = list(df.iloc[row])
        new_lst = [item  for item in new_lst if item is not None]
        #print(new_lst)

        if pre_lst == new_lst:
            continue
            
        for item in df.iloc[row]:
            if item is not None:
                try:
                    if type(float(item)) == float:
                        #print(item, end=", [")
                        f.write("{0}, [".format(item))
                        continue
                except:
                    pass
                #print(item, end=",")
                f.write("{0}, ".format(item))

            else:
                #print("]",end="\n")
                f.write("]\n")
                cnt+=1
                break
        pre_lst = new_lst
f.close()
print(cnt)    

########################END

In [ ]:
#sent = nltk.pos_tag(nltk.word_tokenize("She girl let down her long hair"))
sent = nltk.pos_tag(nltk.word_tokenize("quick"))
print(sent)

#regex grammar
grammar = r"""
NP:{<DT|PRP\$>?<JJ>*<NN>} 
FOO:{<NP><VB.><JJ>}"""
#"NP:{DT|PRP\$>?<JJ>*<NN>}" #, FOO: {<NP><VB.><JJ>}"

#NLTK Regex parser
cp = nltk.RegexpParser(grammar)
tree = cp.parse(sent)
print(tree)
for subtree in tree.subtrees():
    print(subtree)



In [ ]:
#NLTK Regex parser
cp = nltk.RegexpParser(grammar)
tree = cp.parse(sent)
print(tree)
for subtree in tree.subtrees():
    print(subtree)


In [ ]:
for subtree in tree.subtrees():
    if subtree.label() =="NP":
        print(subtree.leaves())

In [ ]:
for sent in sentlist:
    cp.parse(nltk.pos_tag(sent))
    for subtree in tree.subtrees():
        if subtree.label() =="NP":
            print(subtree.leaves())